## Simulating encounters

If this is your first time using a notebook, please ensure you have [Python installed](https://www.python.org/downloads/) and you have installed the additional dependencies with the following command: `pip install -r ./automation/requirements.txt`

First, we'll navigate to the repo's root:


In [1]:
import os

if os.path.basename(os.getcwd()) != "TheGame":
    os.chdir("..")

Next, import what you need:


In [2]:
import copy
from automation.templates.bestiary import Bestiary
from automation.simulator.deck import Deck
from automation.simulator.encounter import Encounter

Use the Bestiary to provide dictionaries for each of the combatants. We can use emojis to represent them in the combat output. The Name value will be shown in the log.

Note that files with SAMPLE in the name are designed to workshop ideas and test edge cases, not be included in the game.


In [3]:
b = Bestiary(input_files=["06_Bestiary_SAMPLE.yaml", "07_PCs.yaml"]).raw_data
c1 = b["Clubs1"]
c1.update(dict(Name="💀", id="A"))
c2 = copy.copy(b["Clubs1"])
c2.update(dict(Name="👽", id="B"))
s = b["Spider Queen"]
s.update(dict(Name="🎇", id="C"))

Initialize the encounter. We can see who is involved and resource information by looking at the encounter's `PCs`, `enemies`, or `turn_order` properties.

Note that this may differ from a real encounter because each participant has their own hand and deck. In a true encounter, a GM might manage many characters with the same deck.


In [4]:
e = Encounter(PCs=[c1, c2], Enemies=[s])
e.turn_order

[56:25][WARNING]: 💀 used 1 excess XP
[56:25][WARNING]: 👽 used 1 excess XP


[💀
 TC       :♠️ 8 | pc.HP : 6/6
 Hand     :  02 | pc.PP : 1/1
 Deck     :  51 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6,
 👽
 TC       :♠️ A | pc.HP : 6/6
 Hand     :  03 | pc.PP : 1/1
 Deck     :  51 | pc.AP : 2/2
 Discards :  00 | RestC : 6/6,
 🎇
 TC       :♠️ A | pc.HP : 11/11
 Hand     :  03 | pc.PP : 6/6
 Deck     :  51 | pc.AP : 1/1
 Discards :  00 | RestC : 11/11]

Next, we can simulate a couple rounds of combat. Here, each participant will choose a Power available to them (if sufficient PP) and apply it to an enemy at random. This does not yet cover buffing powers (e.g., Shield, Lend Aid) or mind control status effects (i.e., Charmed, Enthralled).


In [5]:
e.sim_round(3)

[56:26][INFO]: 💀 used 0/1 PP with Momentum
[56:26][INFO]: ['💀 used Momentum', "🎇 resisted are moved to 1 space of attacker's end position", '💀 wounded 🎇 by 1: AP 0/1, HP 11/11']
[56:26][INFO]: 💀 used 1/1 PP with Shield
[56:26][INFO]: ['💀 used Shield', '💀 wounded 🎇 by 1: AP 0/1, HP 10/11']
[56:26][INFO]: 👽 used 1/1 PP with Shield
[56:26][INFO]: ['👽 used Shield', '👽 wounded 🎇 by 1: AP 0/1, HP 9/11']
[56:26][INFO]: 👽 used 0/0 PP with Momentum
[56:26][INFO]: ['👽 used Momentum', "🎇 are moved to 1 space of attacker's end position", '👽 wounded 🎇 by 1: AP 0/1, HP 8/11']
[56:26][INFO]: 🎇 used 0/6 PP with Terrain Blink
[56:26][INFO]: ['🎇 used Terrain Blink', '🎇 wounded 👽 by 2: AP 0/2, HP 6/6']
[56:26][INFO]: 🎇 used 1/6 PP with Attack, Mystic Aura
[56:26][INFO]: ['🎇 used Attack, Mystic Aura']
[56:26][INFO]: 💀 used 0/0 PP with Momentum
[56:26][INFO]: ['💀 used Momentum', "🎇 resisted are moved to 1 space of attacker's end position"]
[56:26][INFO]: 💀 used 0/0 PP with Weapon Attack
[56:26][INFO]: ['💀 

We can even simulate epic events.


In [6]:
e.sim_epic_event(successes_needed=3)

[56:26][INFO]: Party 1, GM 0 | 👽 Major Success
[56:26][INFO]: Party 1, GM 1 | GM Major Success
[56:26][INFO]: Party 2, GM 1 | 👽 Suited Hit
[56:26][INFO]: Party 2, GM 2 | GM Suited Hit
[56:26][INFO]: Party 3, GM 2 | 💀 Suited Hit
[56:26][INFO]: Party 4, GM 2 | 👽 Suited Hit
[56:26][INFO]: Party 4, GM 3 | GM Suited Hit
[56:26][INFO]: Party wins after 18 total cards drawn


Let's see how everyone is doing.


In [7]:
e.turn_order

[💀
 TC       :♠️ 8 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 0/1
 Deck     :  38 | pc.AP : 2/2
 Discards :  13 | RestC : 6/6,
 👽
 TC       :♠️ A | pc.HP : 5/6
 Hand     :  03 | pc.PP : 0/1
 Deck     :  37 | pc.AP : 0/2
 Discards :  14 | RestC : 6/6,
 🎇
 TC       :♠️ A | pc.HP : 4/11
 Hand     :  06 | pc.PP : 3/6
 Deck     :  39 | pc.AP : 0/1
 Discards :  09 | RestC : 11/11]

If you're interested in draining a specific character's resources, we can index them directly and subtract values.


In [8]:
spider_queen = e.enemies[0]
spider_queen.HP = 1
spider_queen

🎇
TC       :♠️ A | pc.HP : 1/11
Hand     :  06 | pc.PP : 3/6
Deck     :  39 | pc.AP : 0/1
Discards :  09 | RestC : 11/11

The only thing that can't be set directly is the deck. This has to be managed by either drawing, using a fate card, or shuffling.


In [9]:
spider_queen.draw()

♦️ Q

In [10]:
spider_queen.exchange_fate()

[56:26][INFO]: Exchanged Fate Card: ♦️ A


In [11]:
spider_queen.shuffle(2)
spider_queen

🎇
TC       :♥️ 8 | pc.HP : 1/11
Hand     :  05 | pc.PP : 3/6
Deck     :  39 | pc.AP : 0/1
Discards :  10 | RestC : 11/11

We can give some or all participants a rest.


In [12]:
e.sim_quick_rest(participants=e.PCs)  # If no participants specified, all

[56:26][INFO]: 💀 recovered 2 HP/PP/AP during Quick Rest
[56:26][INFO]: 👽 recovered 5 HP/PP/AP during Quick Rest
[56:26][INFO]: 🎇 recovered 15 HP/PP/AP during Quick Rest


In [13]:
e.turn_order

[💀
 TC       :♥️ A | pc.HP : 7/6
 Hand     :  04 | pc.PP : 1/1
 Deck     :  48 | pc.AP : 2/2
 Discards :  02 | RestC : 5/6,
 👽
 TC       :♥️ 4 | pc.HP : 7/6
 Hand     :  03 | pc.PP : 1/1
 Deck     :  45 | pc.AP : 2/2
 Discards :  06 | RestC : 4/6,
 🎇
 TC       :♠️ 8 | pc.HP : 12/11
 Hand     :  05 | pc.PP : 6/6
 Deck     :  42 | pc.AP : 1/1
 Discards :  07 | RestC : 4/11]

Or a full rest.


In [14]:
e.sim_full_rest()

In [15]:
e.turn_order

[💀
 TC       :♦️ K | pc.HP : 6/6
 Hand     :  04 | pc.PP : 1/1
 Deck     :  49 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6,
 👽
 TC       :♥️ 2 | pc.HP : 6/6
 Hand     :  03 | pc.PP : 1/1
 Deck     :  50 | pc.AP : 2/2
 Discards :  01 | RestC : 6/6,
 🎇
 TC       :♦️ 3 | pc.HP : 11/11
 Hand     :  05 | pc.PP : 6/6
 Deck     :  48 | pc.AP : 1/1
 Discards :  01 | RestC : 11/11]

## Logging

Much of the information we're seeing above in the `[TIME][INFO]` format is as a result of the [logger](https://www.loggly.com/ultimate-guide/python-logging-basics/) that keeps track of this information. We can adjust what we see by adjusting the log level.


In [16]:
from automation.utils import logger

logger.setLevel("DEBUG")  # Most information
# logger.setLevel('INFO') # Standard information
# logger.setLevel('WARNING') # Only problems
# logger.setLevel('CRITICAL') # No information
spider_queen.save(DR=3, attrib=["AGL", "STR"], upper_lower="lower", draw_n=2)
e.sim_round()

[56:26][DEBUG]: Drew [♠️ K, ♠️ 4] vs ♦️ 3 with TR 2 at Lower Hand 2: Miss
[56:26][INFO]: 💀 used 0/1 PP with Attack, Vengeance
[56:26][DEBUG]: Drew [♦️ 9] vs ♦️ 3 with TR 4: Suited Miss
[56:26][INFO]: ['💀 used Attack, Vengeance']
[56:26][INFO]: 💀 used 0/1 PP with Momentum
[56:26][DEBUG]: Drew [♦️ K] vs ♦️ 3 with TR 1: Suited Miss
[56:26][DEBUG]: Drew [♠️ K] vs ♦️ 3 with TR 4: Hit
[56:26][INFO]: ['💀 used Momentum', "🎇 are moved to 1 space of attacker's end position", '💀 wounded 🎇 by 1: AP 0/1, HP 11/11']
[56:26][INFO]: 👽 used 0/1 PP with Momentum
[56:26][DEBUG]: Drew [♠️ Q] vs ♦️ 3 with TR 1: Miss
[56:26][DEBUG]: Drew [♠️ T] vs ♦️ 3 with TR 4: Miss
[56:26][INFO]: ['👽 used Momentum', "🎇 are moved to 1 space of attacker's end position"]
[56:26][INFO]: 👽 used 0/1 PP with Attack, Vengeance
[56:26][DEBUG]: Drew [♦️ 4] vs ♦️ 3 with TR 4: Suited Hit
[56:26][INFO]: ['👽 used Attack, Vengeance', '👽 wounded 🎇 by 1: AP 0/1, HP 10/11']
[56:26][INFO]: 🎇 used 0/6 PP with Terrain Blink
[56:26][DEBUG]: D

For a more detailed record of checks, saves, and rests, turn on CSV logging. This will save logs of who performed which check/save or rest and some associated values.


In [17]:
e.set_csv_logging(True)
e.sim_round()
e.sim_quick_rest()

[56:26][INFO]: 💀 used 0/1 PP with Momentum
[56:26][DEBUG]: Drew [♦️ 6] vs ♦️ 3 with TR 1: Suited Miss
[56:26][DEBUG]: Drew [♠️ 5] vs ♦️ 3 with TR 4: Hit
[56:26][INFO]: ['💀 used Momentum', "🎇 are moved to 1 space of attacker's end position", '💀 wounded 🎇 by 1: AP 0/1, HP 9/11']
[56:26][DEBUG]: Drew [♦️ 6] vs ♦️ K with TR 5: Suited Miss
[56:26][INFO]: 💀 remains Entangled
[56:26][INFO]: 👽 used 0/1 PP with Momentum
[56:26][DEBUG]: Drew [♦️ 7] vs ♦️ 3 with TR 1: Suited Miss
[56:26][DEBUG]: Drew [♥️ T] vs ♦️ 3 with TR 4: Color Miss
[56:26][INFO]: ['👽 used Momentum', "🎇 are moved to 1 space of attacker's end position"]
[56:26][INFO]: 👽 used 1/1 PP with Shield
[56:26][DEBUG]: Drew [♠️ 8] vs ♦️ 3 with TR 4: Miss
[56:26][INFO]: ['👽 used Shield']
[56:26][INFO]: 🎇 used 0/5 PP with Terrain Blink
[56:26][DEBUG]: Drew [♠️ 3] vs ♥️ 2 with TR 2: Hit
[56:26][INFO]: ['🎇 used Terrain Blink', '🎇 wounded 👽 by 1: AP 1/2, HP 6/6']
[56:26][INFO]: 🎇 used 1/5 PP with Attack, Mystic Aura
[56:26][DEBUG]: Drew [♠️ 

The following uses system commands rather than Python to look at the output. To use this data more meaningfully, try loading the data with [pandas](https://pythonbasics.org/read-csv-with-pandas/).


In [18]:
!head automation/_output/log_draws.csv

date,id,check_save,result_int,result_str,DR,type,mod,upper_lower,draw_n
04/15 09:55:45,C,save,3,Suited Hit,2,STR,-1,n,1
04/15 09:55:45,B,check,3,Suited Hit,2,Brute,2,n,1
04/15 09:55:45,B,check,3,Suited Hit,2,Brute,2,n,1
04/15 09:55:45,C,save,2,Color Hit,3,None,0,n,1
04/15 09:55:45,C,save,-2,Color Miss,3,None,0,n,1
04/15 09:55:45,C,save,1,Hit,3,None,0,n,1
04/15 09:55:45,A,save,-2,Color Miss,3,None,0,n,1
04/15 09:55:45,B,save,-3,Miss,3,None,0,n,1
04/15 09:56:08,C,save,-3,Miss,2,STR,-1,n,1


In [19]:
!head automation/_output/log_rests.csv

date,name,before_after,type,discards,hand,HP,AP,PP,RestCards
01/22 09:16:38,👽,before,quick,15,3,5,0,1,7
01/22 09:16:38,👽,after,quick,16,5,8,0,2,4
01/22 09:16:38,💀,before,quick,16,5,7,1,0,7
01/22 09:16:38,💀,after,quick,18,5,8,1,2,5
01/22 09:16:38,🎇,before,quick,8,1,1,0,5,8
01/22 09:16:38,🎇,after,quick,13,1,8,0,6,3
01/22 09:16:38,👽,before,full,6,5,8,2,2,4
01/22 09:16:38,👽,after,full,1,5,7,2,2,7
01/22 09:16:38,💀,before,full,8,5,8,2,2,5
